# CSCI 3155 Project 1 - Compile into Stack Machine Bytecode


The objective of this project is to explore (A) compiling arithmetic expressions with let-bindings into a __stack machine bytecode__ and (B) writing an emulator to execute this bytecode. This is all to be done in standalone scala using IntelliJ or some IDE that supports `sbt` (Scala Build Tools).

## Instructions for Testing.

#### IntelliJ

Instructions are provided in a separate pdf/pptx presentation in the same directory as this notebook.
Most students should aim to work with IntelliJ IDE (People have reported that VSCode or Eclipse also work but we do not support them).

#### Command line: sbt
 
You can run tests from the command prompt with the current directory as the very top directory of the project, 
by running the following commands:

```bash
$ sbt compile
$ sbt test
```


### Running ScalaTest tests

Also, we will use a powerful unit testing package called scalatest. The tests themselves are in two files in the directory
`src/test/scala/edu/colorado/csci3155/project1/`

#### IntelliJ

Type `test` in the SBT shell window. It will provide all tests that passed and tests that failed.

#### sbt
To run this go to the terminal and from the very top directory run:

```
$ sbt test
```

It will say success if all tests run or give you a failure messages.

## Instructions for submission
1. Ensure everything is saved.
2. Run the tests one last time to ensure everything works as expected. You will not be able to submit if your code does not at least compile.
3. Run the `checkAndZipSubmission` sbt task
    * (*sbt*) Run `$ sbt checkAndZipSubmission` in a terminal
    * (*IntelliJ*) Type `checkAndZipSubmission` on SBT shell.
4. Ensure whichever option you chose displayed `[success]` at the end
5. Upload the generated "submission.zip" file located at the project root folder (at the same level as "src", "build.sbt", etc.).

**Please Note:** Do not try to zip and submit your entire project folder. It causes trouble with our grading process. 

# The Language
We have encountered arithmetic expressions and let bindings given by the grammar 


$$\begin{array}{rcll}
\textbf{Expr} & \rightarrow & Const(\textbf{Double}) \\
& | & Ident(\textbf{Identifier})\\ 
& | & Plus( \textbf{Expr}, \textbf{Expr})  \\
& | & Minus( \textbf{Expr}, \textbf{Expr}) \\
& | & Mult(\textbf{Expr}, \textbf{Expr}) \\
& | & Div(\textbf{Expr}, \textbf{Expr}) \\
& | & Log(\textbf{Expr}) \\
& | & Exp(\textbf{Expr}) \\
& | & Sine(\textbf{Expr}) \\
& | & Cosine(\textbf{Expr}) \\
& | & Let(\textbf{Identifier}, \textbf{Expr}, \textbf{Expr})\\\\
\textbf{Double} & \rightarrow & \text{all double precision numbers in Scala}\\
\textbf{Identifier} & \rightarrow & \textbf{String} & \text{all scala strings}\\\\
\end{array}$$

The objective of this project is to explore compiling this into a stack machine bytecode and writing an emulator to execute this bytecode.

# Part 1: Stack Machine Bytecode

A stack machine runs instructions that work on a stack of numbers and an environment. The environment binds variable names to numbers and is implemented in the file `Environment.scala` as a list of tuples.  The stack machine has a set of instructions shown below.  Keep the picture below in mind as you think of how a stack machine works.

![Stack Machine Schematic](img/stack-machine-illustration.png)

As an instruction runs, it is going to change the environment and stack.
The following is the specification for stack machine instruction set you will have to implement.
Pay special attention to the `SubIns` and `DivIns` instructions since the order matters.

### Stack Machine Instruction Set 

- `LoadIns(identifier)`: Pop off the top element of the stack. Let the number popped off be `v`. Update the environment by mapping `identifier` to `v`. If the identifier does not already exist a new binding is added, otherwise, the existing binding is shadowed by the new binding.
  - Example: Stack has top element `20.9`. The instruction `LoadIns("x")` removes the top element off the stack and updates the environment by adding the binding `"x" -> 20.9`.  
- `StoreIns(identifier)`: Take the value that `identifier` maps to in the environment (if `identifier` is not present, then throw an exception). Let the value be `v`. Push `v` onto the top of the stack.
  - Example: `StoreIns("what")` will look up the environment for the value `what`, say `4.3`. It will push `4.3` onto the top of stack. Environment is unchanged by this operation. 
- `PushIns(d)`: push the number d onto the stack.
- `PopIns`: pop off the top element of the stack - throw an exception if the stack is empty.
- `AddIns`: pop two numbers from the stack, add them and push the result back to the stack. Throw an exception if the stack is empty during any of the pop operations.
- `SubIns`: pop two numbers from the stack: let the first number popped be `v1` and second number be `v2`, subtract them as `v2 - v1` (this order is very important) and push the result back to the stack. Throw an exception if the stack is empty during any of the pop operations.
- `MultIns`: pop two numbers from the stack, multiply them and push the result back to the stack. Throw an exception if the stack is empty during any of the pop operations.
- `DivIns`: pop two numbers from the stack, let the first number popped be `v1` and second number be `v2`, divide them as `v2 / v1` (this order is very important)  and push the result back to the stack. Throw an exception if the stack is emptyduring any of the pop operations. Throw exception if division by zero.
- `LogIns`: pop _one_ number from the stack, compute its log if positive and push the result back onto the stack. If non-positive throw an exception. Throw an exception if the stack is empty during any of the pop operations.
- `ExpIns`: pop _one_ number from the stack, compute its exponential $e^x$ and push the result back onto the stack.  Throw an exception if the stack is empty during any of the pop operations
- `SineIns/CosineIns`: pop _one_ number from the stack, compute its sin/cos respectively, and push the result back onto the stack.  Throw an exception if the stack is empty during any of the pop operations.

Note: scala has functions: `math.log, math.sin, math.cos and math.exp`.

### Example

Given initial values as follows:
  - List of instructions  `[ StoreIns("x"), PushIns(3.0), AddIns, PushIns(4.0), SubIns, LoadIns("result") ]` and 
  - Environment that binds $[ x \mapsto 2.0, y \mapsto 4.0, z \mapsto 4.0 ]$.
  - Empty Stack (represented by the empty list).

we execute each instruction in turn starting from the empty stack. We will implement the stack as a list with the head of the list as the top of the stack.

- Initially the stack is empty. 
- When we execute  `StoreIns("x")`, the stack is `[ 2.0 ]` and environment is the same.
- When we execute  `PushIns(3.0)`, the stack is `[ 3.0, 2.0 ]` and environment is the same.
- When we execute `AddIns`, the stack becomes `[ 5.0 ]` and environment is the same.
- When we execute `PushIns(4.0)`, the stack becomes `[ 4.0, 5.0 ] ` and environment is the same.
- When we execute `SubIns`, the stack becomes `[ 1.0 ]` and environment is the same.
- When we execute `LoadIns("result")` the stack is now `[]` (empty) and environment is $[ x \mapsto 2.0, y \mapsto 4.0, z \mapsto 4.0, result \mapsto 1.0 ]$.



### Instructions for Part 1

Implement the methods `emulateSingleInstruction` and `emulateStackMachine` in the file `StackMachineEmulator.scala`. For your convenience, the stack machine instructions have been defined as a very simple inductive definition giving case classes for all the instructions. We will use an immutable List data structure to simulate the stack.

- `emulateSingleInstruction(stack, env, instruction)` takes in a `stack`  of type `List[Double]`, an environment of type `Environment.t` (see file `Environment.scala`) and instruction which is of type `Instruction`. It returns a `List[Double]` which represents the stack resulting from executing that instructions and environment `Environment.t` that is the updated environment.

- `emulateStackMachine(listOfInstructions, initialEnvironment)` asks you to __return the final environment__ computed by the `listOfInstructions`. The `initialEnvironment` is passed in from the test cases and is usually `Environment.empty`. You should call `emulateSingleInstruction` repeatedly using functors such as `foldLeft` or using a tail recursive function. Use of loops will result in point deductions. 

#### Coding Style Requirements

  - __The use of while, for loops and mutables var is prohibited for all parts of the project__. 
  - Any recursive functions used for the emulator (part 1 of assignment) must be made `tail recursive` and the annotation `@tailrec` must be used. This however does not apply to part 2 of this assignment below.

# Part 2: Compiling Expressions to a List of ByteCode Instructions

We will now describe the `compilation` of expressions into bytecode instructions.

For instance the expression `let x = 1.0 in x + (2.0 - 3.0) * 5.0` is represented as an AST

~~~
Let("x",
    Const(1.0),
    Plus(Ident("x"), Mult(Minus(Const(2.0), Const(3.0)), Const(5.0))
   )
~~~

The overall goal of this part is to compile this down into a list of bytecode instructions that serves to evaluate this through the emulator you have built in part 1.

For example, the expression above produces the instructions

~~~
PushIns(1.0)
LoadIns("x")
StoreIns("x")
PushIns(2.0)
PushIns(3.0)
MinusIns
PushIns(5.0)
MultIns
AddIns
~~~

You should check that evaluating this sequence gives the result `-4.0`
on top of the stack. Please pay particular attention to the order of the operands for `MinusI` according to the specification provided in Part 1.


The idea is to implement a function __compileExpr(e)__ that given an expression __e__ yields a _list of instructions_ according to the following operational semantics definition.

$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}\;\; (\textit{#3})} $$
$$\newcommand\comp{\textbf{compileExpr}}$$

### Constant Rule

The rule for constants is simple. An expression `Const(f)` compiles to the instruction `PushI(f)`.

$$\semRule{}{\comp(\texttt{Const(f)}) = [ \text{PushIns}(f) ] }{const-rule}$$

Note again that $\comp$ maps expressions to _list_ of instructions.

### Add Rule

$$\semRule{\comp(\texttt{e1}) = L_1,\ \comp(\texttt{e2}) = L_2}{\comp(\texttt{Plus(e1, e2}) = ListContatenation(L_1, L_2 , [ AddIns ]) }{add-rule}$$

The instructions concatenate the lists $L_1, L_2$ along with the list consisting of a single instruction `[ AddI ]`. Note that the `++` operator in scala implements the list concatenation.

### Subtract Rule

$$\semRule{\comp(\texttt{e1}) = L_1,\ \comp(\texttt{e2}) = L_2}{\comp(\texttt{Minus(e1, e2}) = ListContatenation(L_1, L_2 , [ SubIns ]) }{minus-rule}$$

The instructions concatenate the lists $L_1, L_2$ along with the list consisting of a single instruction `[ SubI ]`. 

### Let Rule

$$\semRule{\comp(\texttt{e1}) = L_1,\ \comp(\texttt{e2}) = L_2}{\comp(\texttt{Let("x", e1, e2)}) = 
ListConcatenation(L_1, [LoadIns("x")], L_2)}{let-rule}$$

Notice that the compilation introduces a `LoadI` instruction after executing the instructions $L_1$ corresponding to `e1`. This instruction ensures that the result of the computation is loaded onto the identifier "x" in the environment.

### Ident Rule

`Ident("x")` is simply implemented by the `StoreI` operation.

$$\semRule{}{\comp(\texttt{Ident(str)}) = [StoreIns(str)] }{ident-rule}$$

The rule simply asks you to generate a list with a single instruction for an identifier expression. 

If you have followed the logic clearly, why is this rule not raising any kind of error? Where is the check whether `str` is in the environment beign done??

### Rules for Other expressions

We hope that you will be able to fill in rules for other cases `Mult`, `Div`, `Exp`, `Log`, `Sine` and `Cosine`.



### Instructions for Part 2

The definition of Expression AST is given in the file `Expr.scala`

Your goal is to implement the compilation routine `compileToStackMachineCode(e: Expr): List[StackMachineInstruction]` in the file `StackMachineCompilation.scala`. The function takes in an expression `e` and outputs a list of stack machine instructions.